<a href="https://colab.research.google.com/github/yezzzzin/object-detectinon/blob/main/AiProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. 라이브러리 import

In [18]:
!pip install torch torchvision

In [19]:
import pandas as pd
import torch
import os
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from torch.optim.lr_scheduler import StepLR

In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

2. 데이터 로드 & 전처리

In [21]:
# 데이터 로드 및 분할
data = pd.read_csv('/content/drive/MyDrive/dataset.csv')

train_data = pd.DataFrame()
test_data = pd.DataFrame()

for label in data['라벨'].unique():
    label_data = data[data['라벨'] == label]
    train_label_data, test_label_data = train_test_split(label_data, test_size=200, random_state=42)  # Splitting
    train_data = train_data.append(train_label_data)
    test_data = test_data.append(test_label_data)

train_data = train_data.sample(frac=1).reset_index(drop=True)
test_data = test_data.sample(frac=1).reset_index(drop=True)

print(train_data[:10])

                                               데이터위치        라벨
0        /content/drive/MyDrive/img/dog/dog.4213.jpg       dog
1      /content/drive/MyDrive/img/lamb/lamb.438.jpeg      lamb
2  /content/drive/MyDrive/img/squirrel/squirrel.7...  squirrel
3    /content/drive/MyDrive/img/horse/horse.204.jpeg     horse
4  /content/drive/MyDrive/img/elephant/elephant.7...  elephant
5    /content/drive/MyDrive/img/horse/horse.666.jpeg     horse
6    /content/drive/MyDrive/img/horse/horse.623.jpeg     horse
7    /content/drive/MyDrive/img/horse/horse.273.jpeg     horse
8        /content/drive/MyDrive/img/dog/dog.4966.jpg       dog
9    /content/drive/MyDrive/img/horse/horse.757.jpeg     horse


<ipython-input-21-408b67e455ef>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data = train_data.append(train_label_data)
<ipython-input-21-408b67e455ef>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_data = test_data.append(test_label_data)
<ipython-input-21-408b67e455ef>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data = train_data.append(train_label_data)
<ipython-input-21-408b67e455ef>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_data = test_data.append(test_label_data)
<ipython-input-21-408b67e455ef>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a 

In [22]:
# 사용자 정의 데이터셋 클래스
class AnimalDataset(Dataset):
    def __init__(self, dataframe, label_map, transform=None):
        self.dataframe = dataframe
        self.label_map = label_map
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        image_path = self.dataframe.iloc[idx, 0]
        image = Image.open(image_path).convert("RGB")  # tensor 단위 오류 해결 위해 명시
        label_name = self.dataframe.iloc[idx, 1]
        label = self.label_map[label_name]

        if self.transform:
            image = self.transform(image)

        return image, label

# 라벨을 정수로 매핑 - 추후 label을 숫자가 아닌 글자로 반환하도록 해주는 딕셔너리
label_map = {'cat': 0, 'cow': 1, 'dog': 2, 'elephant': 3, 'horse': 4, 'lamb': 5, 'squirrel': 6}

# 이미지 전처리를 위한 transform 함수 정의
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 데이터셋 및 데이터 로더 생성
train_dataset = AnimalDataset(train_data, label_map, transform=transform)  # 인스턴스는 init 파라미터 따름
test_dataset = AnimalDataset(test_data, label_map, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)  # cpu 내에서 병렬처리해줌. 4까지는 가능
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

3. 훈련

In [23]:
# 모델 정의
model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)  # imageNet 데이터로 훈련된 모델 불러옴

num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 7)  # 7종의 동물 클래스

model = model.to(device)

# 손실 함수와 옵티마이저 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)

# 학습 함수
def train_model(model, criterion, optimizer, num_epochs=25):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        # 진행률 표시를 위해 tqdm을 사용
        for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            inputs, labels = inputs.to(device), labels.to(device)  # 데이터를 GPU로 이동

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        epoch_loss = running_loss / len(train_loader.dataset)
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")

        scheduler.step()

    return model

In [24]:
# 모델 학습
trained_model = train_model(model, criterion, optimizer, num_epochs=20)

Epoch 1/20: 100%|██████████| 175/175 [00:37<00:00,  4.73it/s]


Epoch 1/20, Loss: 0.0201


Epoch 2/20: 100%|██████████| 175/175 [00:34<00:00,  5.08it/s]


Epoch 2/20, Loss: 0.0066


Epoch 3/20: 100%|██████████| 175/175 [00:37<00:00,  4.65it/s]


...

Epoch 18/20, Loss: 0.0031


Epoch 19/20: 100%|██████████| 175/175 [00:34<00:00,  5.02it/s]


Epoch 19/20, Loss: 0.0031


Epoch 20/20: 100%|██████████| 175/175 [00:37<00:00,  4.71it/s]

Epoch 20/20, Loss: 0.0034


In [25]:
# 테스트 함수
def test_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)  # 데이터를 GPU로 이동

            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'테스트 데이터에 대한 모델 정확도: {accuracy:.2f}%')

# 모델 테스트
test_model(trained_model, test_loader)

테스트 데이터에 대한 모델 정확도: 92.36%
